In [1]:
import re
def clean_str(text):
    search = ["أ","إ","آ","ة","_","-","/",".","،"," و "," يا ",'"',"ـ","'","ى","\\",'\n', '\t','&quot;','?','؟','!']
    replace = ["ا","ا","ا","ه"," "," ","","",""," و"," يا","","","","ي","",' ', ' ',' ',' ? ',' ؟ ',' ! ']

    # Remove tashkeel
    p_tashkeel = re.compile(r'[\u0617-\u061A\u064B-\u0652]')
    text = re.sub(p_tashkeel,"", text)

    # Remove longation
    p_longation = re.compile(r'(.)\1+')
    subst = r"\1\1"
    text = re.sub(p_longation, subst, text)

    text = text.replace('وو', 'و')
    text = text.replace('يي', 'ي')
    text = text.replace('اا', 'ا')

    for i in range(0, len(search)):
        text = text.replace(search[i], replace[i])

    # Trim
    text = text.strip()

    return text



In [3]:
import json
import csv

# Specify the paths to your input JSON file and output CSV file
input_json_file_path = '/content/labels.json'
output_csv_file_path = 'file.csv'

# Load data from the JSON file
with open(input_json_file_path, 'r', encoding='utf-8') as json_file:
    data = json.load(json_file)

# Create a CSV file and write header
with open(output_csv_file_path, 'w', encoding='utf-8', newline='') as csv_file:
    csv_writer = csv.writer(csv_file)
    csv_writer.writerow(['text', 'label'])

    # Iterate through each entry in the JSON file
    for entry in data:
        text = entry.get('text', '')
        labels = entry.get('label', [])

        # Extract relevant information and write to CSV
        for label_info in labels:
            start = label_info.get('start', '')
            end = label_info.get('end', '')
            label_text = label_info.get('text', '')
            label_names = ', '.join(label_info.get('labels', []))

            row = [f'{text[start:end]} ({label_text})', label_names]
            csv_writer.writerow(row)

print(f"CSV file created successfully at: {output_csv_file_path}")


CSV file created successfully at: file.csv


In [4]:
import pandas as pd
df1 = pd.read_csv('/content/file.csv')
df1.head()

,text,label
0,احط (احط),add
1,عنوان ( عنوان),address
2,الفاتورة ( الفاتورة),invoice
3,اغير ( اغير ),change
4,عنوان (عنوان),address


In [5]:
csv_file_path = "/content/ner_data.csv"

df2 = pd.read_csv(csv_file_path)

In [6]:
df1['text'] = df1['text'].str.replace(r'\([^)]*\)', '')

# Save the modified dataframe to a new CSV file
# df.to_csv('modified_dataset.csv', index=False)

<ipython-input-6-ff16296f8a84>:1: FutureWarning: The default value of regex will change from True to False in a future version.
  df1['text'] = df1['text'].str.replace(r'\([^)]*\)', '')


In [7]:
df1.head()

,text,label
0,احط,add
1,عنوان,address
2,الفاتورة,invoice
3,اغير,change
4,عنوان,address


In [8]:
df2

,text,label
0,عايزة احط عنوان بيتي,X X X X
1,ابغى الفاتورة من فضلك,X X X X
2,عايزة اغير عنوان البيت,X X X X
3,ممكن اغير عنواني,X X X
4,عايز منيو ارابياتا جامعة الدول,X X X X X
...,...,...
5025,ضيف على الاوردر 3 بيج ماك كومبو لارج,X X X itemnumber ItemName ItemName X ItemSize
5026,حضرتك ممكن تغيرلي الرقم المتسجل عندكم,X X X X X X
5027,وش هي طريقة حذف الجوال القديم,X X X X X X
5028,فاضل قداش وقت عالاوردر,X X X X


In [ ]:
# df1.to_csv('cleaned_labels.csv')

In [9]:
df2.head()

,text,label
0,عايزة احط عنوان بيتي,X X X X
1,ابغى الفاتورة من فضلك,X X X X
2,عايزة اغير عنوان البيت,X X X X
3,ممكن اغير عنواني,X X X
4,عايز منيو ارابياتا جامعة الدول,X X X X X


In [10]:
data1 = df1.copy()
data2 = df2.copy()
# Step 1: Split the text column in the second dataset into words
data2['words'] = data2['text'].apply(lambda x: x.split())

In [11]:
data2.head()

,text,label,words
0,عايزة احط عنوان بيتي,X X X X,"[عايزة, احط, عنوان, بيتي]"
1,ابغى الفاتورة من فضلك,X X X X,"[ابغى, الفاتورة, من, فضلك]"
2,عايزة اغير عنوان البيت,X X X X,"[عايزة, اغير, عنوان, البيت]"
3,ممكن اغير عنواني,X X X,"[ممكن, اغير, عنواني]"
4,عايز منيو ارابياتا جامعة الدول,X X X X X,"[عايز, منيو, ارابياتا, جامعة, الدول]"


In [12]:
data1.head()

,text,label
0,احط,add
1,عنوان,address
2,الفاتورة,invoice
3,اغير,change
4,عنوان,address


In [13]:
data1.head()

,text,label
0,احط,add
1,عنوان,address
2,الفاتورة,invoice
3,اغير,change
4,عنوان,address


In [14]:
data1['text'] = data1['text'].str.replace(r'\s', '') # remove indent before & after each splitted text

<ipython-input-14-38d97a929dee>:1: FutureWarning: The default value of regex will change from True to False in a future version.
  data1['text'] = data1['text'].str.replace(r'\s', '') # remove indent before & after each splitted text


In [15]:
data2.head()

,text,label,words
0,عايزة احط عنوان بيتي,X X X X,"[عايزة, احط, عنوان, بيتي]"
1,ابغى الفاتورة من فضلك,X X X X,"[ابغى, الفاتورة, من, فضلك]"
2,عايزة اغير عنوان البيت,X X X X,"[عايزة, اغير, عنوان, البيت]"
3,ممكن اغير عنواني,X X X,"[ممكن, اغير, عنواني]"
4,عايز منيو ارابياتا جامعة الدول,X X X X X,"[عايز, منيو, ارابياتا, جامعة, الدول]"


In [ ]:
import pandas as pd



# Creating a new dataset with 'new_text' and 'new_label' columns
new_data = pd.DataFrame(columns=['new_text', 'new_label'])

# Loop through each row in the second dataset
for index, row in data2.iterrows():
    new_text = []
    new_label = []

    # Loop through each word in the current row
    for word in row['words']:
        # Check if the word exists in the first dataset
        match = data1[data1['text'] == word]

        # If a match is found, append the corresponding text and label to the new_text and new_label lists
        if not match.empty:
            new_text.append(word)
            new_label.append(match['label'].values[0])

    # Add a new row to the new dataset with the concatenated new_text and new_label lists
    new_data = new_data.append({'new_text': ' '.join(new_text), 'new_label': ' '.join(new_label)}, ignore_index=True)

# Display the new dataset
print(new_data)


In [17]:
new_data.isnull().sum()

new_text     0
new_label    0
dtype: int64

In [20]:
new_data.tail()

,new_text,new_label
5025,الاوردر 3,order item_number
5026,تغيرلي,change
5027,حذف الجوال,delete phone_number
5028,وقت,late
5029,,


In [ ]:
# new_data.to_csv('final_data.csv')